In [1]:
import datetime as dt
import inspect
import os
import sys

import ee
from IPython.display import Image
from importlib import reload  # Python 3 only
# from imp import reload      # Python 2/3 via futures modules

import interpolate
# Assume ET models will be installed via pip or available in EE
# For now, add parent folder to path in order to access models in other folders
# This seems super awful, is there a better way to do this in the short term?
root_path = os.path.dirname(os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))))
if os.path.join(root_path, 'ndvi-et-ee') not in sys.path:
    sys.path.insert(0, os.path.join(root_path, 'ndvi-et-ee'))
from ndvi_et import ndvi_et

interpolate = reload(interpolate)
ndvi_et = reload(ndvi_et)

ee.Initialize()

et_palette = [
  'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
  '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

In [2]:
# Input parameters

# Date range you want to aggregate ET over
start_date = '2015-08-01'
end_date = '2015-09-15'

cloud_cover = 70
interp_days = 32
interp_type = 'LINEAR'

study_area = ee.Geometry.Rectangle(-119.25, 38.80, -119.05, 39.15)
study_region = study_area.coordinates().getInfo()

In [3]:
# Add extra Landsat images at start and end to interpolate between
interp_start_date = (dt.datetime.strptime(start_date, '%Y-%m-%d') - dt.timedelta(days=interp_days)).strftime('%Y-%m-%d')
interp_end_date = (dt.datetime.strptime(end_date, '%Y-%m-%d') + dt.timedelta(days=interp_days)).strftime('%Y-%m-%d')
print(interp_start_date)
print(interp_end_date)

2015-06-30
2015-10-17


In [4]:
# Build the input Landsat collection
landsat_coll = ee.ImageCollection([])
landsat_coll = ee.ImageCollection(landsat_coll.merge(
    ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA') \
        .filterDate(interp_start_date, interp_end_date) \
        .filterBounds(study_area) \
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', cloud_cover) \
        .filterMetadata('DATA_TYPE', 'equals', 'L1TP')))
# landsat_coll = ee.ImageCollection(landsat_coll.merge(
#     ee.ImageCollection('LANDSAT/LE07/C01/T1_RT_TOA') \
#         .filterDate(interp_start_date, interp_end_date) \
#         .filterBounds(study_area) \
#         .filterMetadata('CLOUD_COVER_LAND', 'less_than', cloud_cover) \
#         .filterMetadata('DATA_TYPE', 'equals', 'L1TP')))
# landsat_coll = ee.ImageCollection(landsat_coll.merge(
#     ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
#         .filterDate(interp_start_date, interp_end_date) \
#         .filterBounds(study_area) \
#         .filterMetadata('CLOUD_COVER_LAND', 'less_than', cloud_cover) \
#         .filterMetadata('DATA_TYPE', 'equals', 'L1TP')))
print(landsat_coll.aggregate_histogram('system:index').getInfo())

{'2_LC08_042033_20150713': 1, '2_LC08_042033_20150729': 1, '2_LC08_042033_20150814': 1, '2_LC08_042033_20150830': 1, '2_LC08_042033_20150915': 1, '2_LC08_043033_20150720': 1, '2_LC08_043033_20150805': 1, '2_LC08_043033_20150821': 1, '2_LC08_043033_20150906': 1, '2_LC08_043033_20150922': 1, '2_LC08_043033_20151008': 1}


In [5]:
Image(url=ee.Image(landsat_coll.first()).select([3, 2, 1]) \
    .getThumbURL({'min': 0.0, 'max': 0.3, 'region': study_region}))

In [6]:
# Compute ETf for each Landsat scene
def compute_et_fraction(image):
    s = ndvi_et.NDVI_ET.fromLandsatTOA(
        toa_image=ee.Image(image), m=1.25, b=0)
    return ee.Image(s.etf())
scene_et_fraction_coll = ee.ImageCollection(
    landsat_coll.map(compute_et_fraction))
print(ee.Image(scene_et_fraction_coll.first()).getInfo())

{'type': 'Image', 'bands': [{'id': 'etf', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -1.25, 'max': 1.25}, 'dimensions': [7811, 7941], 'crs': 'EPSG:32611', 'crs_transform': [30.0, 0.0, 245385.0, 0.0, -30.0, 4425915.0]}], 'properties': {'system:time_start': 1436812371360, 'system:index': '2_LC08_042033_20150713'}}


In [7]:
# Compute ETf for each Landsat scene
def compute_et_fraction(image):
    s = ndvi_et.NDVI_ET.fromLandsatTOA(
        toa_image=ee.Image(image), m=1.25, b=0)
    return ee.Image(s.etf())
scene_et_fraction_coll = ee.ImageCollection(
    landsat_coll.map(compute_et_fraction))

# Daily reference ET collection
daily_et_reference_coll = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET') \
    .filterDate(start_date, end_date) \
    .select(['etr'], ['et_reference'])

# Compute composite/mosaic images for each image date
daily_et_fraction_coll = ee.ImageCollection(interpolate.aggregate_daily(
    image_coll=scene_et_fraction_coll,
    start_date=interp_start_date,
    end_date=interp_end_date))

# Interpolate daily ETf, multiply by daily ETr, and sum to ET
daily_et_actual_coll = ee.ImageCollection(interpolate.interp_et_coll(
    et_reference_coll=daily_et_reference_coll,
    et_fraction_coll=daily_et_fraction_coll,
    interp_days=interp_days,
    interp_type=interp_type))

In [8]:
Image(url=ee.Image(daily_et_actual_coll.sum()) \
    .getThumbURL({'min': 0.0, 'max': 250, 'region': study_region, 'palette': ','.join(et_palette)}))